In [1]:
import numpy as np

import torch
from torch import nn
from torch.nn import functional as F

import zmq
import threading

In [2]:
class Responder(threading.Thread):
    def __init__(self):
        super().__init__()
        self.a = threading.Event()
        self.b = threading.Event()
    
    def reset(self):
        self.a.clear()
        self.b.clear()
        
    def run(self):
        while True:
            self.a.wait()
            self.a.clear()
            
            self.b.set()

In [3]:
thread = Responder()
thread.start()

In [4]:
%%timeit 
thread.b.clear()
thread.a.set()
thread.b.wait()

29.9 µs ± 3.07 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [5]:
context = zmq.Context()

In [6]:
class ResponderZMQ(threading.Thread):
    def __init__(self, context):
        super().__init__()
        self.a = context.socket(zmq.PAIR)
        self.a.bind("inproc://A")
        
    def run(self):
        while True:
            self.a.recv()
            self.a.send(b'D')

In [7]:
thread2 = ResponderZMQ(context)
thread2.start()

In [8]:
aa = context.socket(zmq.PAIR)
aa.connect("inproc://A")



In [9]:
%%timeit 
aa.send(b'P')
aa.recv()

17.4 µs ± 343 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [13]:
import random

In [17]:
aa.send_json()

In [28]:
import secrets
keys = [(secrets.token_bytes(4), random.randint(0, 32)) for _ in range(64)]

In [29]:
from itertools import chain

In [32]:
fk = list(chain.from_iterable(keys))

In [22]:
import dill, pickle, json, msgpack

In [18]:
%timeit pickle.dumps(keys)

6.28 µs ± 193 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [19]:
%timeit dill.dumps(keys)

412 µs ± 4.11 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [25]:
%timeit msgpack.dumps(keys)

8.03 µs ± 55 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [33]:
%timeit msgpack.dumps(fk)

5 µs ± 156 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [34]:
%timeit pickle.dumps(fk)

3.5 µs ± 41.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [35]:
a = [1, 2]

In [36]:
x, y = map(lambda x: x + 1, a)

In [38]:
y

3